In [1]:
import nni.common.blob_utils
nni.common.blob_utils.NNI_BLOB = 'https://repo.dailylime.kr/mirror/nni'

In [2]:
import nni
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10
from nni.nas.evaluator.pytorch import DataLoader

CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
])
valid_data = nni.trace(CIFAR10)(root='./data', train=False, download=True, transform=transform_valid)
valid_loader = DataLoader(valid_data, batch_size=256, num_workers=6)


Files already downloaded and verified


In [3]:
from nni.nas.hub.pytorch import DARTS as DartsSpace

darts_v2_model = DartsSpace.load_searched_model('darts-v2', pretrained=True, download=True)

def evaluate_model(model, cuda=False):
    device = torch.device('cuda' if cuda else 'cpu')
    model.to(device)
    model.eval()
    with torch.no_grad():
        correct = total = 0
        for inputs, targets in valid_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            logits = model(inputs)
            _, predict = torch.max(logits, 1)
            correct += (predict == targets).sum().cpu().item()
            total += targets.size(0)
    print('Accuracy:', correct / total)
    return correct / total

evaluate_model(darts_v2_model, cuda=True)  # Set this to false if there's no GPU.


/usr/local/lib/python3.8/dist-packages/nni/nas/profiler/pytorch/utils/shape_formula.py:107: UserWarning: Cannot find a default in torch.ops.aten because <built-in method relu of PyCapsule object at 0x7fd7c6c936c0> has no attribute default. Skip registering the shape inference formula.
  warnings.warn(f'Cannot find a {name} in torch.ops.aten because {object} has no attribute {name}. '
/usr/local/lib/python3.8/dist-packages/nni/nas/profiler/pytorch/utils/shape_formula.py:107: UserWarning: Cannot find a default in torch.ops.aten because <built-in method gelu of PyCapsule object at 0x7fd7c6c93b40> has no attribute default. Skip registering the shape inference formula.
  warnings.warn(f'Cannot find a {name} in torch.ops.aten because {object} has no attribute {name}. '
/usr/local/lib/python3.8/dist-packages/nni/nas/profiler/pytorch/utils/shape_formula.py:107: UserWarning: Cannot find a default in torch.ops.aten because <built-in method hardswish of PyCapsule object at 0x7fd7c6c938d0> has no 

[2024-01-24 16:58:29] "/root/.cache/nni/nashub/darts-v2-5465b0d2.pth" already exists. Checking hash.
Accuracy: 0.9737


0.9737

In [4]:
model_space = DartsSpace(
    width=16,           # the initial filters (channel number) for the model
    num_cells=8,        # the number of stacked cells in total
    dataset='cifar'     # to give a hint about input resolution, here is 32x32
)


In [5]:
import numpy as np
from nni.nas.evaluator.pytorch import Classification
from torch.utils.data import SubsetRandomSampler

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
])

train_data = nni.trace(CIFAR10)(root='./data', train=True, download=True, transform=transform)

num_samples = len(train_data)
indices = np.random.permutation(num_samples)
split = num_samples // 2

search_train_loader = DataLoader(
    train_data, batch_size=64, num_workers=6,
    sampler=SubsetRandomSampler(indices[:split]),
)

search_valid_loader = DataLoader(
    train_data, batch_size=64, num_workers=6,
    sampler=SubsetRandomSampler(indices[split:]),
)

evaluator = Classification(
    learning_rate=1e-3,
    weight_decay=1e-4,
    train_dataloaders=search_train_loader,
    val_dataloaders=search_valid_loader,
    max_epochs=10,
    gpus=1,
    fast_dev_run=True,
    num_classes=10
)


Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_test_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [6]:
from nni.nas.strategy import DARTS as DartsStrategy

strategy = DartsStrategy()

In [7]:
from nni.nas.experiment import NasExperiment

experiment = NasExperiment(model_space, evaluator, strategy)
experiment.run()


[2024-01-24 16:58:47] Config is not provided. Will try to infer.
[2024-01-24 16:58:47] Strategy is found to be a one-shot strategy. Setting execution engine to "sequential" and format to "raw".
[2024-01-24 16:58:47] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-01-24 16:58:47] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-01-24 16:58:47] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-01-24 16:58:52] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-01-24 16:58:52] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-01-24 16:58:52] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-01-24 16:58:52] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-01-24 16:58:52] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-01-24 16

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

  | Name            | Type                 | Params
---------------------------------------------------------
0 | training_module | ClassificationModule | 2.0 M 
---------------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
7.950     Total estimated model params size (MB)
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

[2024-01-24 16:59:13] Waiting for models submitted to engine to finish...
[2024-01-24 16:59:13] Experiment is completed.
[2024-01-24 16:59:13] WARNING: `training_service` will be ignored for sequential execution engine.


True

In [8]:
exported_arch = experiment.export_top_models(formatter='dict')[0]

exported_arch

[2024-01-24 16:59:13] Sorted weights in differentiable cell export (normal cell, node 2): [(0.001859489013440907, 1, 'skip_connect'), (0.0018236733740195632, 0, 'dil_conv_5x5'), (0.0014605340547859669, 0, 'avg_pool_3x3'), (0.0010679308325052261, 0, 'sep_conv_5x5'), (0.0009991331025958061, 1, 'sep_conv_3x3'), (0.0006362202693708241, 0, 'dil_conv_3x3'), (0.0005384317482821643, 0, 'skip_connect'), (0.00024037530238274485, 1, 'max_pool_3x3'), (1.9297374819871038e-05, 0, 'max_pool_3x3'), (-1.6804688129923306e-05, 1, 'sep_conv_5x5'), (-0.0006067798240110278, 0, 'sep_conv_3x3'), (-0.0010500650387257338, 1, 'dil_conv_5x5'), (-0.0012172305723652244, 1, 'dil_conv_3x3'), (-0.0016504812519997358, 1, 'avg_pool_3x3')]
[2024-01-24 16:59:13] Sorted weights in differentiable cell export (normal cell, node 3): [(0.002360126469284296, 2, 'max_pool_3x3'), (0.002219046000391245, 1, 'sep_conv_3x3'), (0.002118476666510105, 0, 'dil_conv_3x3'), (0.002080781850963831, 0, 'sep_conv_3x3'), (0.001574288704432547, 

{'normal/op_2_0': 'skip_connect',
 'normal/input_2_0': [1],
 'normal/op_2_1': 'dil_conv_5x5',
 'normal/input_2_1': [0],
 'normal/op_3_0': 'max_pool_3x3',
 'normal/input_3_0': [2],
 'normal/op_3_1': 'sep_conv_3x3',
 'normal/input_3_1': [1],
 'normal/op_4_0': 'dil_conv_3x3',
 'normal/input_4_0': [2],
 'normal/op_4_1': 'max_pool_3x3',
 'normal/input_4_1': [1],
 'normal/op_5_0': 'max_pool_3x3',
 'normal/input_5_0': [1],
 'normal/op_5_1': 'sep_conv_5x5',
 'normal/input_5_1': [2],
 'reduce/op_2_0': 'sep_conv_5x5',
 'reduce/input_2_0': [1],
 'reduce/op_2_1': 'max_pool_3x3',
 'reduce/input_2_1': [0],
 'reduce/op_3_0': 'dil_conv_5x5',
 'reduce/input_3_0': [1],
 'reduce/op_3_1': 'avg_pool_3x3',
 'reduce/input_3_1': [0],
 'reduce/op_4_0': 'skip_connect',
 'reduce/input_4_0': [1],
 'reduce/op_4_1': 'avg_pool_3x3',
 'reduce/input_4_1': [2],
 'reduce/op_5_0': 'skip_connect',
 'reduce/input_5_0': [2],
 'reduce/op_5_1': 'max_pool_3x3',
 'reduce/input_5_1': [0]}

In [9]:
from nni.nas.space import model_context

with model_context(exported_arch):
    final_model = DartsSpace(width=16, num_cells=8, dataset='cifar')
    
train_loader = DataLoader(train_data, batch_size=96, num_workers=6)  # Use the original training data

max_epochs = 100

evaluator = Classification(
    learning_rate=1e-3,
    weight_decay=1e-4,
    train_dataloaders=train_loader,
    val_dataloaders=valid_loader,
    max_epochs=max_epochs,
    gpus=1,
    export_onnx=False,          # Disable ONNX export for this experiment
    fast_dev_run=True,   # Should be false for fully training
    num_classes=10,
)

evaluator.fit(final_model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_test_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | metrics   | ModuleDict       | 0     
2 | _model    | DARTS            | 219 K 
-----------------------------------------------
21

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[2024-01-24 16:59:19] Intermediate result: 0.09375  (Index 0)
[2024-01-24 16:59:19] Final result: 0.09375
